# Kunitz Domain Detection Using Profile Hidden Markov Models (HMM)

This notebook implements a bioinformatics pipeline to identify the Kunitz-type protease inhibitor domain (PF00014) in protein sequences using HMM-based detection.

We will:
- Collect and preprocess positive and negative datasets.
- Reduce redundancy using CD-HIT and extract structural alignments from PDB.
- Build an HMM profile using `hmmbuild`.
- Filter sequences to avoid bias in evaluation.
- Run `hmmsearch` on test sets.
- Evaluate model performance with multiple thresholds.

## Linux Command-Line Tools: Summary of Usage

This section summarizes the most commonly used Linux shell commands in this notebook:

- **`cat`**: Concatenates and displays the content of files.
- **`awk`**: A powerful text-processing tool used for pattern scanning and field-based extraction.
- **`grep`**: Searches for lines matching a pattern within a file.
- **`cut`**: Extracts specific columns or fields from each line of a file.
- **`sort`**: Sorts lines in a file, alphabetically or numerically.
- **`comm`**: Compares two sorted files and shows common and distinct lines.
- **`makeblastdb`**: Creates a searchable BLAST database from a FASTA file.
- **`blastp`**: Compares protein sequences using BLAST against a protein database.
- **`cd-hit`**: Clusters sequences to reduce redundancy based on sequence identity thresholds.
- **`hmmbuild`**: Builds a profile Hidden Markov Model (HMM) from a multiple sequence alignment.
- **`hmmsearch`**: Searches sequence databases for matches to a profile HMM.
- **`head` / `tail`** – Outputs the first or last N lines of a file (useful for dataset splitting).
- **`wc`** – Word count; used with `-l` to count lines (e.g., number of sequences).
- **`less`** – Paginates the display of long files (useful for inspection).

## 2. Extract Human and Non-Human Kunitz Sequences

From the file `all_kunitz.fasta`, extract sequences annotated as "Homo sapiens" and separate them from non-human sequences.

In [ ]:
# Estrai sequenze umane
awk '/^>/ {f=($0 ~ /Homo sapiens/)} f' all_kunitz.fasta > human_kunitz.fasta

# Estrai sequenze non umane
grep -v "Homo sapiens" all_kunitz.fasta > nothuman_kunitz.fasta

# Controllo del numero di sequenze umane (intestazioni)
grep "sp" human_kunitz.fasta | wc -l


## 3. Extract PF00014 sequences from PDB report and cluster with CD-HIT

Download a CSV from RCSB PDB filtered for:
- PFAM = PF00014
- Length 45–80 aa
- Resolution ≤ 3.5 Å

Then extract the sequences in FASTA and reduce redundancy using CD-HIT at 90%.


In [ ]:
# Estrai sequenze PF00014 dal CSV
cat rcsb_pdb_custom_report_20250410062557.csv | tr -d '"' \
| awk -F ',' '{if (length($2)>0) {name=$2}; print name ,$3,$4,$5}' \
| grep PF00014 \
| awk '{print ">"$1"_"$3; print $2}' > pdb_kunitz_customreported.fasta

# Rimuovi ridondanza con CD-HIT (90%)
cd-hit -i pdb_kunitz_customreported.fasta -o pdb_kunitz_customreported_nr.fasta -c 0.9


## 4. Structural Alignment and HMM Building

Use the representative sequences to submit to PDBeFold and download the `.ali` alignment. Reformat for `hmmbuild`, then build the structural HMM.


In [ ]:
# Riformattazione del file .ali in FASTA-like
awk '{
  if (substr($1,1,1)==">") {
    print "\n" toupper($1)
  } else {
    printf "%s", toupper($1)
  }
}' pdb_kunitz_rp.ali > pdb_kunitz_rp_formatted.ali

# Costruzione dell'HMM
hmmbuild structural_model.hmm pdb_kunitz_rp_formatted.ali


## 5. BLAST Filtering to Remove Overlapping Sequences

To avoid evaluating the same sequences used to build the model, we perform BLAST between the representative PDB Kunitz sequences and the full UniProt Kunitz set.

We retain only sequences with identity < 95% and coverage < 50%.


In [ ]:
# Crea il database BLAST da tutte le sequenze Kunitz (umane + non umane)
makeblastdb -in all_kunitz.fasta -dbtype prot -out all_kunitz.fasta

# Lancia BLAST
blastp -query pdb_kunitz_rp.fasta -db all_kunitz.fasta -out pdb_kunitz_nr_23.blast -outfmt 7


In [ ]:
# Estrai gli ID UniProt da escludere (identity ≥ 95% e coverage ≥ 50%)
grep -v "^#" pdb_kunitz_nr_23.blast | awk '{if ($3>=95 && $4>=50) print $2}' | sort -u | cut -d "|" -f 2 > to_remove.ids

# Estrai tutti gli ID delle sequenze Kunitz
grep ">" all_kunitz.fasta | cut -d "|" -f 2 > all_kunitz.id

# Ottieni gli ID da mantenere
comm -23 <(sort all_kunitz.id) <(sort to_remove.ids) > to_keep.ids


In [ ]:
# Estrai le sequenze finali da tenere (positivi validi per il test)
python3 get_seq.py to_keep.ids all_kunitz.fasta ok_kunitz.fasta


## 6. Construction of the Negative Dataset

Extract from SwissProt all proteins that do NOT contain the Kunitz domain and create the negative FASTA dataset.


In [ ]:
# Ottieni tutti gli ID da SwissProt
grep ">" uniprot_sprot.fasta | cut -d "|" -f 2 > sp.id

# Escludi gli ID che appartengono a sequenze Kunitz
comm -23 <(sort sp.id) <(sort all_kunitz.id) > sp_negs.ids

# Estrai le sequenze negative finali
python3 get_seq.py sp_negs.ids uniprot_sprot.fasta sp_negs.fasta


## 7. Train/Test Set Construction

Split the positive and negative datasets into training and testing halves using randomization.


In [ ]:
# Randomizza gli ID
sort -R to_keep.ids > random_ok_kunitz.ids
sort -R sp_negs.ids > random_sp_negs.ids

# Dividi in due metà
head -n 183 random_ok_kunitz.ids > pos_1.ids
tail -n 183 random_ok_kunitz.ids > pos_2.ids

head -n 286417 random_sp_negs.ids > neg_1.ids
tail -n 286417 random_sp_negs.ids > neg_2.ids


In [ ]:
# Estrai i FASTA dei 4 set
python3 get_seq.py pos_1.ids uniprot_sprot.fasta > pos_1.fasta
python3 get_seq.py pos_2.ids uniprot_sprot.fasta > pos_2.fasta
python3 get_seq.py neg_1.ids uniprot_sprot.fasta > neg_1.fasta
python3 get_seq.py neg_2.ids uniprot_sprot.fasta > neg_2.fasta


## 8. HMMER Search and .class File Generation

We run `hmmsearch` on each of the four FASTA sets (pos_1, pos_2, neg_1, neg_2) using the trained HMM.  
To ensure consistency in e-value calculation across datasets of different sizes, we use the `-Z 1000` option.  
We convert the output to `.class` format to later evaluate classification performance.


In [ ]:
# Esegui hmmsearch con tabular output per ogni set
hmmsearch -Z 1000 --max --tblout pos_1.out structural_model.hmm pos_1.fasta
hmmsearch -Z 1000 --max --tblout pos_2.out structural_model.hmm pos_2.fasta
hmmsearch -Z 1000 --max --tblout neg_1.out structural_model.hmm neg_1.fasta
hmmsearch -Z 1000 --max --tblout neg_2.out structural_model.hmm neg_2.fasta


We now extract useful information from the HMMER output:
- Identifier
- True label (1 for positive, 0 for negative)
- Full-sequence E-value
- Domain E-value (if available)

These are stored in `.class` files to be used with the `performance.py` script.


In [ ]:
# POSITIVI
grep -v "^#" pos_1.out | awk '{split($1,a,"|"); print a[2]"\t1\t"$5"\t"$8}' > pos_1.class
grep -v "^#" pos_2.out | awk '{split($1,a,"|"); print a[2]"\t1\t"$5"\t"$8}' > pos_2.class

# NEGATIVI
grep -v "^#" neg_1.out | awk '{split($1,a,"|"); print a[2]"\t0\t"$5"\t"$8}' > neg_1.class
grep -v "^#" neg_2.out | awk '{split($1,a,"|"); print a[2]"\t0\t"$5"\t"$8}' > neg_2.class


If `hmmsearch` does not report a match, that sequence is still part of the dataset and should be considered a true negative.  
We assign them a default E-value of 10.0 and append them to the `.class` files using `comm`.


In [ ]:
# NEGATIVI NON MATCHATI: li aggiungiamo con E-value 10.0
comm -23 <(sort neg_1.ids) <(cut -f1 neg_1.class | sort) | awk '{print $1"\t0\t10.0\t10.0"}' >> neg_1_hits.class
comm -23 <(sort neg_2.ids) <(cut -f1 neg_2.class | sort) | awk '{print $1"\t0\t10.0\t10.0"}' >> neg_2_hits.class


## 9. Evaluate Model Performance with Different E-value Thresholds

We now merge the positive and negative `.class` files into two evaluation sets (`set_1` and `set_2`) and use the script `performance.py` to compute evaluation metrics such as:
- Accuracy
- Precision
- Recall
- F1 score
- MCC (Matthews Correlation Coefficient)

We repeat the evaluation at multiple E-value thresholds (e.g., 1e-3 to 1e-10) to identify the optimal decision boundary.


In [ ]:
# Unione dei set
cat pos_1.class neg_1_hits.class > set_1.class
cat pos_2.class neg_2_hits.class > set_2.class


## 10. Evaluate Model Performance at Different Thresholds and Analyze Errors

In this step, we evaluate model performance at different E-value thresholds using `performance.py`. The goal is to identify the optimal threshold based on the **Matthews Correlation Coefficient (MCC)**.

We also analyze false negatives—true positive sequences with high E-values that were misclassified as negatives. This helps us identify potential sensitivity issues in the model.

- Run performance evaluation for both `set_1.class` and `set_2.class` at a fixed threshold (`1e-5`)
- Repeat evaluation across multiple thresholds (from `1e-1` to `1e-10`)
- Sort results based on MCC to find the best cutoff
- Extract and save the worst false negatives for comparison and further inspection


In [ ]:
# Calcola performance su soglia fissa
python3 performance.py set_1.class 1e-5
python3 performance.py set_2.class 1e-5

# Ripeti per diversi e-value per trovare la soglia ottimale (in base a MCC)
for i in $(seq 1 10); do
  python3 performance.py set_1.class 1e-$i
done | sort -nrk 6  # Ordina in base alla sesta colonna (MCC)

# Analisi errori – Trova falsi negativi con E-value alti
sort -grk 3 pos_1.class | less

# Estrai i peggiori falsi negativi in file separati per confronto
awk '$2 == 1 && $3 > 1e-5' pos_1.class | sort -grk 3 > fn_pos1.txt
awk '$2 == 1 && $3 > 1e-5' pos_2.class | sort -grk 3 > fn_pos2.txt